In [16]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Tiingo API Key
API_KEY = '03aaf0f08452293fe7a1259cc00c88b61ee57780'  # Replace with your actual API key

In [6]:
def get_trading_data(ticker, start_date, end_date):
    # Construct the API URL
    headers = {'Content-Type': 'application/json'}
    url = f"https://api.tiingo.com/iex/{ticker}/prices?startDate={start_date}&endDate={end_date}&resampleFreq=5min&afterHours=false&columns=open,high,low,close,volume&token={API_KEY}"
#     params = {
#         'token': API_KEY,
#         'startDate': start_date,
#         'endDate': end_date,
#         'resampleFreq': '5min',
#         'afterHours': 'false',
#         'columns': ['open','high','low','close','volume']
#     }

    # Make the request
    
    response = requests.get(url, headers=headers)
    data = response.json()
    return data


def filter_trading_hours(data):
    trading_data = []
    for item in data:
        # Parse and convert the timestamp to datetime object with timezone info
        timestamp = datetime.fromisoformat(item['date'].replace('Z', '+00:00')).astimezone()
        
        # Filter for active trading hours (9:30 AM to 4:00 PM New York time)
        if timestamp.hour >= 9 and timestamp.minute >= 30 and timestamp.hour < 16:
            # Remove timezone information
            item['date'] = timestamp.replace(tzinfo=None).isoformat()
            trading_data.append(item)
    return trading_data

def get_last_n_trading_days(ticker, n_days):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=10)  # Look back 10 days to ensure 3 trading days are covered

    raw_data = get_trading_data(ticker, start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))
    print(raw_data)
    if type(raw_data) != list or raw_data == []:
        return None
    filtered_data = filter_trading_hours(raw_data)

    # Convert to DataFrame
    df = pd.DataFrame(filtered_data)
    df['date'] = pd.to_datetime(df['date'])

    # Ensure only the last n trading days are included
    trading_days = df['date'].dt.date.unique()[-n_days:]
    df = df[df['date'].dt.date.isin(trading_days)]

    # Remove timezone information if any remains
    df['date'] = df['date'].dt.tz_localize(None)
    df.insert(1, 'ticker', ticker)
    
    return df


In [17]:
#Get all the tickers for F500 listed in the US Stock Exchange
file_path = r"F500.csv"
f500 = pd.read_csv(file_path)
tickers_f500 = list(f500["Symbol"][51:101])
print(tickers_f500)

['AMGN', 'TBB', 'AXP', 'ISRG', 'NEE', 'PFE', 'SPGI', 'CAT', 'DIS', 'RTX', 'MS', 'T', 'UNP', 'CMCSA', 'GS', 'AMAT', 'PGR', 'UBER', 'LOW', 'SYK', 'LMT', 'TJX', 'BLK', 'HON', 'BKNG', 'ELV', 'REGN', 'PLD', 'BSX', 'VRTX', 'COP', 'NKE', 'MDT', 'SCHW', 'MMC', 'ADP', 'AMT', 'PANW', 'UPS', 'C', 'ADI', 'DE', 'SBUX', 'SIRI', 'KKR', 'ANET', 'BX', 'GILD', 'MDLZ', 'CI']


In [29]:
tickers = [
    "AAPL"
#     , "MSFT", "AMZN", "GOOGL", "FB", "TSLA", "BRK.B", "NVDA", "JPM", "JNJ",
#     "V", "PG", "UNH", "HD", "DIS", "MA", "PYPL", "NFLX", "ADBE", "INTC",
#     "VZ", "CSCO", "PEP", "T", "KO", "MRK", "PFE", "ABT", "NKE", "ORCL",
#     "CRM", "XOM", "LLY", "WMT", "QCOM", "BAC", "ABBV", "COST", "MCD", "AMD",
    # "MDT", "HON", "AMGN", "NEE", "LOW", "BA", "AVGO", "UPS", "TXN", "UNP",
    # "CVX", "IBM", "DHR", "SBUX", "BMY", "MMM", "GS", "RTX", "BLK", "LIN",
    # "GE", "ISRG", "CAT", "SPGI", "TMO", "INTU", "PLD", "MO", "MS", "SCHW",
    # "LMT", "TJX", "AMT", "EL", "DE", "ADP", "SYK", "MDLZ", "ZTS", "C",
    # "BKNG", "NOW", "AXP", "CB", "ADI", "TGT", "MMC", "CSX", "DUK", "GILD",
    # "CCI", "ADSK", "FIS", "ICE", "MU", "SO", "REGN", "EW", "CL", "EQIX"
]
print(len(tickers))

1


In [18]:
df = pd.DataFrame()

for ticker in tickers_f500:
    df_ticker = get_last_n_trading_days(ticker, 3)
    if df_ticker is not None:
        df = pd.concat([df, df_ticker], ignore_index=False)

print(df.head)

def save_to_excel(df, filename):
    # Save DataFrame to Excel
    df.to_excel(filename, index=False)


[{'date': '2024-09-06T13:30:00.000Z', 'open': 324.93, 'high': 325.65, 'low': 324.585, 'close': 325.65, 'volume': 354.0}, {'date': '2024-09-06T13:35:00.000Z', 'open': 325.65, 'high': 325.65, 'low': 325.61, 'close': 325.61, 'volume': 5.0}, {'date': '2024-09-06T13:40:00.000Z', 'open': 325.61, 'high': 326.37, 'low': 325.61, 'close': 325.96, 'volume': 146.0}, {'date': '2024-09-06T13:45:00.000Z', 'open': 325.895, 'high': 326.245, 'low': 325.895, 'close': 326.245, 'volume': 102.0}, {'date': '2024-09-06T13:50:00.000Z', 'open': 326.075, 'high': 326.16, 'low': 325.51, 'close': 325.51, 'volume': 233.0}, {'date': '2024-09-06T13:55:00.000Z', 'open': 325.47, 'high': 325.47, 'low': 324.865, 'close': 325.03, 'volume': 402.0}, {'date': '2024-09-06T14:00:00.000Z', 'open': 325.11, 'high': 325.54, 'low': 324.855, 'close': 324.855, 'volume': 280.0}, {'date': '2024-09-06T14:05:00.000Z', 'open': 324.855, 'high': 324.855, 'low': 324.6, 'close': 324.76, 'volume': 121.0}, {'date': '2024-09-06T14:10:00.000Z', 'o

In [19]:
with open('Trading data.csv','r') as infile:
    df.to_csv('Trading data.csv',mode='a',encoding='utf-8')

In [9]:
# df.to_csv('Trading data.csv', encoding='utf-8')

In [31]:
print(df_ticker.columns)

Index(['date', 'ticker', 'open', 'high', 'low', 'close', 'volume'], dtype='object')
